In [1]:
import json
import random
from collections import Counter

In [2]:
import sys
sys.path.append("../../../ner/BERT-NER/")

In [6]:
from bert import Ner

model = Ner("../../../ner/BERT-NER/out/")

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [17]:
def get_ents(sent):
    output =  model.predict(sent)
    ents = []
    ent = None
    for k, v in output:
        assert k is not None, k
        if v['tag'] == 'O' and ent is not None:
            ents.append(ent.lower())
            ent = None
        if v['tag'].startswith('B'):
            if ent is not None:
                ents.append(ent.lower())
            ent = k
        if v['tag'].startswith('I') or v['tag'].startswith('X'):
            if ent is None:
                ent = k
            elif k.startswith("'"):
                ent += k
            else:
                ent += ' ' + k
    return ents

# def get_ents(sent):
#     output =  model.predict(sent)
#     ents = {}
#     ent = None
#     for k, v in output.items():
#         assert k is not None, k
#         if v['tag'] != 'O':
#             ents[k.lower()] = v['tag']
#     return ents

In [8]:
questions = []
contexts = []
examples = []
labels = []
id2idx = {}
files = ["../../dataset/train-v2.0.json",  "../../dataset/dev-v2.0.json"]
for file in files:
    with open(file, 'r') as handle:
        jdata = json.load(handle)
        data = jdata['data']
    for i in range(len(data)):
        section = data[i]['paragraphs']
        for sec in section:
            context = sec['context']
            contexts.append(context)
            qas = sec['qas']
            for j in range(len(qas)):
                question = qas[j]['question']
                is_imp = qas[j]['is_impossible']
                qid = qas[j]['id']
                questions.append(question)
                labels.append(is_imp)
                examples.append((len(contexts)-1, len(questions)-1))
                id2idx[qid] = len(questions)-1

In [11]:
get_ents(questions[0])

['beyonce']

In [34]:
sampled_examples_ans = []
sampled_examples_unans = []
for e in random.sample(examples, len(examples)):
    if labels[e[1]] == True and len(sampled_examples_unans) < 50:
        sampled_examples_unans.append(e)
    elif labels[e[1]] == False and len(sampled_examples_ans) < 50:
        sampled_examples_ans.append(e)

In [35]:
sampled_examples_ans[0]

(3950, 23281)

In [36]:
c,q = sampled_examples_ans[0]
print(contexts[c])
print(questions[q])

Prior to 1871, the government of the United States regularly entered into treaties with Native Americans but the Indian Appropriations Act of March 3, 1871 (ch. 120, 16 Stat. 563) had a rider (25 U.S.C. § 71) attached that effectively ended the President’s treaty making by providing that no Indian nation or tribe shall be acknowledged as an independent nation, tribe, or power with whom the United States may contract by treaty. The federal government continued to provide similar contractual relations with the Indian tribes after 1871 by agreements, statutes, and executive orders.
By what did the US federal government provide contractual relations with Indian tribes following the Indian Appropriations Act of March 3, 1871?


In [25]:
for c,q in sampled_examples_ans:
    context = contexts[c]
    question = questions[q]
    cents = get_ents(context)
    qents = get_ents(question)
    printed = False
    for ent in qents:
        if ent in cents:
            print(question)
            print("~"*20)
            print(ent)
            print("~"*20)
            printed = True
    if printed:
        print("#"*20)

What leader took power in Sudan and was inspired by Nasser?
~~~~~~~~~~~~~~~~~~~~
sudan
~~~~~~~~~~~~~~~~~~~~
What leader took power in Sudan and was inspired by Nasser?
~~~~~~~~~~~~~~~~~~~~
nasser
~~~~~~~~~~~~~~~~~~~~
####################
Who did Nigel Lythgoe say was a fan favorite for the majority of the season?
~~~~~~~~~~~~~~~~~~~~
nigel lythgoe
~~~~~~~~~~~~~~~~~~~~
####################
How much does Barcelona donate to UNICEF per year?
~~~~~~~~~~~~~~~~~~~~
barcelona
~~~~~~~~~~~~~~~~~~~~
How much does Barcelona donate to UNICEF per year?
~~~~~~~~~~~~~~~~~~~~
unicef
~~~~~~~~~~~~~~~~~~~~
####################
What happened to the nationalists and Marxists that had helped the Islamic ruling government to depose the Shah?
~~~~~~~~~~~~~~~~~~~~
marxists
~~~~~~~~~~~~~~~~~~~~
What happened to the nationalists and Marxists that had helped the Islamic ruling government to depose the Shah?
~~~~~~~~~~~~~~~~~~~~
islamic
~~~~~~~~~~~~~~~~~~~~
What happened to the nationalists and Marxists that had h

In [27]:
for c,q in sampled_examples_unans:
    context = contexts[c]
    question = questions[q]
    cents = get_ents(context)
    qents = get_ents(question)
    printed = False
    for ent in qents:
        if ent in cents:
            print(context)
            print(question)
            print("~"*20)
            print(ent)
            print("~"*20)
            printed = True
    if printed:
        print("#"*20)

Director Sima Urale is an award-winning filmmaker. Urale's short film O Tamaiti won the prestigious Best Short Film at the Venice Film Festival in 1996. Her first feature film Apron Strings opened the 2008 NZ International Film Festival. The feature film Siones Wedding, co-written by Oscar Kightley, was financially successful following premieres in Auckland and Apia. The 2011 film The Orator was the first ever fully Samoan film, shot in Samoa in the Samoan language with a Samoan cast telling a uniquely Samoan story. Written and directed by Tusi Tamasese, it received much critical acclaim and attention at film festivals throughout the world.
Where did the film Apron Strings receive critical acclaim and attention in 2011?
~~~~~~~~~~~~~~~~~~~~
apron strings
~~~~~~~~~~~~~~~~~~~~
####################
Roman histories and biographies anticipated the extensive mediaeval literature of lives of saints and miraculous chronicles, but the most characteristic form of the Middle Ages was the romance,

The propagation of universities was not necessarily a steady progression, as the 17th century was rife with events that adversely affected university expansion. Many wars, and especially the Thirty Years' War, disrupted the university landscape throughout Europe at different times. War, plague, famine, regicide, and changes in religious power and structure often adversely affected the societies that provided support for universities. Internal strife within the universities themselves, such as student brawling and absentee professors, acted to destabilize these institutions as well. Universities were also reluctant to give up older curricula, and the continued reliance on the works of Aristotle defied contemporary advancements in science and the arts. This era was also affected by the rise of the nation-state. As universities increasingly came under state control, or formed under the auspices of the state, the faculty governance model (begun by the University of Paris) became more and m

Some NPOs which are particularly well known, often for the charitable or social nature of their activities performed during a long period of time, include Amnesty International, Oxfam, Rotary International, Kiwanis International, Carnegie Corporation of New York, Nourishing USA, DEMIRA Deutsche Minenräumer (German Mine Clearers), FIDH International Federation for Human Rights, Goodwill Industries, United Way, ACORN (now defunct), Habitat for Humanity, Teach For America, the Red Cross and Red Crescent organizations, UNESCO, IEEE, INCOSE, World Wide Fund for Nature, Heifer International, Translators Without Borders and SOS Children's Villages.
What has happened to Teach for America now that its non functional?
~~~~~~~~~~~~~~~~~~~~
teach for america
~~~~~~~~~~~~~~~~~~~~
####################
In March's Super Tuesday, Kerry won decisive victories in the California, Connecticut, Georgia, Maryland, Massachusetts, New York, Ohio, and Rhode Island primaries and the Minnesota caucuses. Dean, des

# Statistics:
1- How many question entities are in the context by percent?  
2- Above split by ans and unans  
&nbsp;&nbsp;&nbsp;&nbsp;        -- Between 50 and 60% each, no obvious split  
3- Those that are not copied, find 50 and label reason  
4- Of those that do appear, how many appear multiple times  

In [38]:
len(sampled_examples_ans), len(sampled_examples_unans)

(50, 50)

In [46]:
sampled_examples_ans = []
sampled_examples_unans = []
for e in random.sample(examples, len(examples)):
    if labels[e[1]] == True and len(sampled_examples_unans) < 100:
        sampled_examples_unans.append(e)
    elif labels[e[1]] == False and len(sampled_examples_ans) < 100:
        sampled_examples_ans.append(e)

counts = Counter()
for c,q in sampled_examples_unans:
    context = contexts[c]
    question = questions[q]
    cents = get_ents(context)
    qents = get_ents(question)    
    for ent in qents:
        counts[ent in cents] += 1
        if cents.count(ent) > 1:
            counts["UNANS"]+=1
print("Unanswerable", counts[True]/ (sum(counts.values())))
print("multi-unans", counts["UNANS"] / counts[True])

counts = Counter()
for c,q in sampled_examples_ans:
    context = contexts[c]
    question = questions[q]
    cents = get_ents(context)
    qents = get_ents(question)    
    for ent in qents:
        counts[ent in cents] += 1
        if cents.count(ent) > 1:
            counts["ANS"]+=1
print("Answerable", counts[True]/ (sum(counts.values())))
print("multi-ans", counts["ANS"] / counts[True])

Unanswerable 0.5151515151515151
multi-unans 0.0
Answerable 0.6785714285714286
multi-ans 0.0


In [52]:
sampled_examples_ans = []
sampled_examples_unans = []
for e in random.sample(examples, len(examples)):
    if labels[e[1]] == True and len(sampled_examples_unans) < 100:
        sampled_examples_unans.append(e)
    elif labels[e[1]] == False and len(sampled_examples_ans) < 100:
        sampled_examples_ans.append(e)
        
for c,q in sampled_examples_unans:
    context = contexts[c]
    question = questions[q]
    cents = get_ents(context)
    for ent in cents:
        if cents.count(ent) > 1:
            print("FOUND ONE")
            
for c,q in sampled_examples_ans:
    context = contexts[c]
    question = questions[q]
    cents = get_ents(context)
    for ent in cents:
        if cents.count(ent) > 1:
            print("FOUND ONE")

FOUND ONE
FOUND ONE


In [53]:
sampled_examples_ans = []
sampled_examples_unans = []
for e in random.sample(examples, len(examples)):
    if labels[e[1]] == True and len(sampled_examples_unans) < 100:
        sampled_examples_unans.append(e)
    elif labels[e[1]] == False and len(sampled_examples_ans) < 100:
        sampled_examples_ans.append(e)

no_copy_unans = []
for c,q in sampled_examples_unans:
    context = contexts[c]
    question = questions[q]
    cents = get_ents(context)
    qents = get_ents(question)    
    for ent in qents:
        if ent not in cents:
            no_copy_unans.append((context, question, ent))

no_copy = []
counts = Counter()
for c,q in sampled_examples_ans:
    context = contexts[c]
    question = questions[q]
    cents = get_ents(context)
    qents = get_ents(question)    
    for ent in qents:
        if ent not in cents:
            no_copy.append((context, question, ent))


In [55]:
for i in range(10):
    c, q, e = no_copy[i]
    print(c)
    print("~"*20)
    print(q)
    print("~"*20)
    print(e)
    print("#"*20)

Cairo University is ranked as 401-500 according to the Academic Ranking of World Universities (Shanghai Ranking) and 551-600 according to QS World University Rankings. American University in Cairo is ranked as 360 according to QS World University Rankings and Al-Azhar University, Alexandria University and Ain Shams University fall in the 701+ range. Egypt is currently opening new research institutes for the aim of modernising research in the nation, the most recent example of which is Zewail City of Science and Technology.
~~~~~~~~~~~~~~~~~~~~
According to QS World University Rankings and Al-Azhar University, Alexandria University and Ain Shams University are all ranked where?
~~~~~~~~~~~~~~~~~~~~
qs world university rankings
####################
On 8 September 1945, U.S. Lt. Gen. John R. Hodge arrived in Incheon to accept the Japanese surrender south of the 38th parallel. Appointed as military governor, General Hodge directly controlled South Korea as head of the United States Army Mi

Answerable reasons:  
not extracted correctly: 8 (included sub ents eg. last name only)  
morphological change: 1   
minor misspellings: 1 (hyphens and capitaliation and special characters)  

In [61]:
for i in range(10):
    c, q, e = no_copy_unans[i]
    print(c)
    print("~"*20)
    print(q)
    print("~"*20)
    print(e)
    print("#"*20)

According to this school of Vedanta, all reality is Brahman, and there exists nothing whatsoever which is not Brahman. Its metaphysics includes the concept of māyā and ātman. Māyā connotes "that which exists, but is constantly changing and thus is spiritually unreal". The empirical reality is considered as always changing and therefore "transitory, incomplete, misleading and not what it appears to be". The concept of ātman is of soul, self within each person, each living being. Advaita Vedantins assert that ātman is same as Brahman, and this Brahman is within each human being and all life, all living beings are spiritually interconnected, and there is oneness in all of existence. They hold that dualities and misunderstanding of māyā as the spiritual reality that matters is caused by ignorance, and are the cause of sorrow, suffering. Jīvanmukti (liberation during life) can be achieved through Self-knowledge, the understanding that ātman within is same as ātman in another person and all 

Answerable reasons:    
not extracted correctly: 4 (included sub ents eg. last name only)    
morphological change: 2 (plurals)     
minor misspellings: 1 (hyphens and capitaliation and special characters)    
wrong ent: 1 (eg. non-)  
real pharaphrase: 1 (eg. dnc, liverpool FC,   

# TODO
- fix ent extraction and matching

In [65]:
sampled_examples_ans = []
sampled_examples_unans = []
for e in random.sample(examples, len(examples)):
    if labels[e[1]] == True and len(sampled_examples_unans) < 100:
        sampled_examples_unans.append(e)
    elif labels[e[1]] == False and len(sampled_examples_ans) < 100:
        sampled_examples_ans.append(e)

counts = Counter()
for c,q in sampled_examples_unans:
    context = contexts[c]
    question = questions[q]
    #cents = get_ents(context)
    qents = get_ents(question)    
    for ent in qents:
        counts[ent in context.lower()] += 1
        if context.lower().count(ent) > 1:
            counts["UNANS"]+=1
print("Unanswerable", counts[True]/ (sum(counts.values())))
print("multi-unans", counts["UNANS"] / counts[True])

counts = Counter()
for c,q in sampled_examples_ans:
    context = contexts[c]
    question = questions[q]
    #cents = get_ents(context)
    qents = get_ents(question)    
    for ent in qents:
        counts[ent in context.lower()] += 1
        if context.lower().count(ent) > 1:
            counts["ANS"]+=1
print("Answerable", counts[True]/ (sum(counts.values())))
print("multi-ans", counts["ANS"] / counts[True])

Unanswerable 0.6203703703703703
multi-unans 0.40298507462686567
Answerable 0.5478260869565217
multi-ans 0.6666666666666666


# results
Unanswerable 0.5698924731182796  
multi-unans 0.37735849056603776  
Answerable 0.6260162601626016  
multi-ans 0.5064935064935064  

Unanswerable 0.6203703703703703  
multi-unans 0.40298507462686567  
Answerable 0.5478260869565217  
multi-ans 0.6666666666666666  

In [75]:
sampled_examples_ans = []
sampled_examples_unans = []
for e in random.sample(examples, len(examples)):
    if labels[e[1]] == True and len(sampled_examples_unans) < 100:
        sampled_examples_unans.append(e)
    elif labels[e[1]] == False and len(sampled_examples_ans) < 100:
        sampled_examples_ans.append(e)

no_copy_unans = []
for c,q in sampled_examples_unans:
    context = contexts[c]
    question = questions[q]
    qents = get_ents(question)    
    for ent in qents:
        if ent not in context.lower():
            no_copy_unans.append((context, question, ent))

no_copy = []
counts = Counter()
for c,q in sampled_examples_ans:
    context = contexts[c]
    question = questions[q]
    qents = get_ents(question)    
    for ent in qents:
        if ent not in context.lower():
            no_copy.append((context, question, ent))

In [76]:
for i in range(10):
    c, q, e = no_copy_unans[i]
    print(c)
    print("~"*20)
    print(q)
    print("~"*20)
    print(e)
    print("#"*20)

A variety of games for the FDS were released by Nintendo (including some like Super Mario Bros. which had already been released on cartridge) and third party companies such as Konami and Taito. A few unlicensed titles were made as well. However, its limitations became quickly apparent as larger ROM chips were introduced, allowing cartridges with greater than 128k of space. More advanced memory management chips (MMC) soon appeared and the FDS quickly became obsolete. Nintendo also charged developers considerable amounts of money to produce FDS games, and many refused to develop for it, instead continuing to make cartridge titles. Many FDS disks have no dust covers (except in some unlicensed and bootleg variants) and are easily prone to getting dirt on the media. In addition, the drive use a belt which breaks frequently and requires invasive replacement. After only two years, the FDS was discontinued, although vending booths remained in place until 1993 and Nintendo continued to service 

Unanswerable:
fake: 6
synonym: 2 (brand + item, x + system, 
morphological: 1

In [77]:
for i in range(10):
    c, q, e = no_copy[i]
    print(c)
    print("~"*20)
    print(q)
    print("~"*20)
    print(e)
    print("#"*20)

The early Manchu rulers also established two foundations of legitimacy which help to explain the stability of their dynasty. The first was the bureaucratic institutions and the neo-Confucian culture which they adopted from earlier dynasties. Manchu rulers and Han Chinese scholar-official elites gradually came to terms with each other. The examination system offered a path for ethnic Han to become officials. Imperial patronage of Kangxi Dictionary demonstrated respect for Confucian learning, while the Sacred Edict of 1670 effectively extolled Confucian family values. The second major source of stability was the Central Asian aspect of their Manchu identity which allowed them to appeal to Mongol, Tibetan and Uighur constituents. The Qing used the title of Emperor (Huangdi) in Chinese while among Mongols the Qing monarch was referred to as Bogda khan (wise Khan), and referred to as Gong Ma in Tibet. Qianlong propagated the image of himself as Buddhist sage rulers, patrons of Tibetan Buddh

Answerable:
external synonym: 0.5 jesuit vs christian
morphological: 2 plurals
parse error: 2
minor misspellings: 1 special characters
outside knowledge: 1.5
missing reference: 1
paraphrase: 2

In [13]:
context

'The pound-force has a metric counterpart, less commonly used than the newton: the kilogram-force (kgf) (sometimes kilopond), is the force exerted by standard gravity on one kilogram of mass. The kilogram-force leads to an alternate, but rarely used unit of mass: the metric slug (sometimes mug or hyl) is that mass that accelerates at 1 m·s−2 when subjected to a force of 1 kgf. The kilogram-force is not a part of the modern SI system, and is generally deprecated; however it still sees use for some purposes as expressing aircraft weight, jet thrust, bicycle spoke tension, torque wrench settings and engine output torque. Other arcane units of force include the sthène, which is equivalent to 1000 N, and the kip, which is equivalent to 1000 lbf.'

In [15]:
model.predict(contexts[0])

[('Beyoncé', {'tag': 'B-PER', 'confidence': 0.9998725652694702}),
 ('Giselle', {'tag': 'I-PER', 'confidence': 0.9964303374290466}),
 ('Knowles-Carter', {'tag': 'I-PER', 'confidence': 0.9865139126777649}),
 ('(', {'tag': 'O', 'confidence': 0.9998637437820435}),
 ('/biːˈjɒnseɪ/', {'tag': 'O', 'confidence': 0.9971987009048462}),
 ('bee-YON-say', {'tag': 'O', 'confidence': 0.9991575479507446}),
 (')', {'tag': 'O', 'confidence': 0.997826874256134}),
 ('(', {'tag': 'O', 'confidence': 0.9990881681442261}),
 ('born', {'tag': 'O', 'confidence': 0.9967760443687439}),
 ('September', {'tag': 'O', 'confidence': 0.7898686528205872}),
 ('4', {'tag': 'O', 'confidence': 0.9509610533714294}),
 (',', {'tag': 'O', 'confidence': 0.8050832748413086}),
 ('1981', {'tag': 'O', 'confidence': 0.9888269901275635}),
 (')', {'tag': 'O', 'confidence': 0.9879037737846375}),
 ('is', {'tag': 'O', 'confidence': 0.9998726844787598}),
 ('an', {'tag': 'O', 'confidence': 0.9991201758384705}),
 ('American', {'tag': 'B-MISC',

In [19]:
get_ents(contexts[0])

['beyoncé giselle knowles-carter',
 'american',
 'houston',
 'texas',
 'r & b',
 "destiny's child",
 'mathew knowles',
 'beyoncé',
 'dangerously in love',
 'grammy awards',
 'billboard hot 100',
 'crazy in love',
 'baby boy']